In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, roc_auc_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.ensemble import VotingClassifier
from tqdm import tqdm
import re
from sklearn.preprocessing import LabelEncoder
import pickle
import os
import torch
import platform
from sentence_transformers import SentenceTransformer

In [3]:
!nvidia-smi

Mon Dec  1 05:43:30 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.95.05              Driver Version: 580.95.05      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        On  |   00000000:42:00.0 Off |                  N/A |
| 37%   45C    P8             14W /  575W |       0MiB /  32607MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

# Data Analysis

In [4]:
rule_col = "rule"
body_col = "body"  
label_col = "rule_violation"

In [5]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")  
solution_df = pd.read_csv("data/solution.csv")

In [7]:
train_df.info()
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2029 entries, 0 to 2028
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              2029 non-null   int64 
 1   body                2029 non-null   object
 2   rule                2029 non-null   object
 3   subreddit           2029 non-null   object
 4   positive_example_1  2029 non-null   object
 5   positive_example_2  2029 non-null   object
 6   negative_example_1  2029 non-null   object
 7   negative_example_2  2029 non-null   object
 8   rule_violation      2029 non-null   int64 
dtypes: int64(2), object(7)
memory usage: 142.8+ KB


,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2,rule_violation
0,0,Banks don't want you to know this! Click here ...,"No Advertising: Spam, referral links, unsolici...",Futurology,If you could tell your younger self something ...,hunt for lady for jack off in neighbourhood ht...,Watch Golden Globe Awards 2017 Live Online in ...,"DOUBLE CEE x BANDS EPPS - ""BIRDS""\n\nDOWNLOAD/...",0
1,1,SD Stream [ ENG Link 1] (http://www.sportsstre...,"No Advertising: Spam, referral links, unsolici...",soccerstreams,[I wanna kiss you all over! Stunning!](http://...,LOLGA.COM is One of the First Professional Onl...,#Rapper \n🚨Straight Outta Cross Keys SC 🚨YouTu...,[15 Amazing Hidden Features Of Google Search Y...,0
2,2,Lol. Try appealing the ban and say you won't d...,No legal advice: Do not offer or request legal...,pcmasterrace,Don't break up with him or call the cops. If ...,It'll be dismissed: https://en.wikipedia.org/w...,Where is there a site that still works where y...,Because this statement of his is true. It isn'...,1
3,3,she will come your home open her legs with an...,"No Advertising: Spam, referral links, unsolici...",sex,Selling Tyrande codes for 3€ to paypal. PM. \n...,tight pussy watch for your cock get her at thi...,NSFW(obviously) http://spankbang.com/iy3u/vide...,Good News ::Download WhatsApp 2.16.230 APK for...,1
4,4,code free tyrande --->>> [Imgur](http://i.imgu...,"No Advertising: Spam, referral links, unsolici...",hearthstone,wow!! amazing reminds me of the old days.Well...,seek for lady for sex in around http://p77.pl/...,must be watch movie https://sites.google.com/s...,We're streaming Pokemon Veitnamese Crystal RIG...,1


In [8]:
test_df.info()
test_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54059 entries, 0 to 54058
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   row_id              54059 non-null  int64 
 1   body                54059 non-null  object
 2   rule                54059 non-null  object
 3   subreddit           54059 non-null  object
 4   positive_example_1  54059 non-null  object
 5   positive_example_2  54059 non-null  object
 6   negative_example_1  54059 non-null  object
 7   negative_example_2  54059 non-null  object
dtypes: int64(1), object(7)
memory usage: 3.3+ MB


,row_id,body,rule,subreddit,positive_example_1,positive_example_2,negative_example_1,negative_example_2
0,2029,Hi Ken l! /u/Whimsical_wispers told me he want...,No promotion of illegal activity: Do not encou...,IAmA,There are several web sites on internet that a...,Someone needs to rape that judge and see how w...,call whores watch for you herein http://onj.me...,52 http://MySexFind.com - find girl for sex no...
1,2030,Doubt anyone will read this but anti depressan...,No medical advice: Do not offer or request spe...,science,Maybe he just has alzheimer's. It would explai...,Hmm well I WAS going to ask women of Reddit ho...,You could always just carry your kid and get t...,Maybe you should get started on the solution a...
2,2031,**[One of her best show](http://shortora.com/i...,"No Advertising: Spam, referral links, unsolici...",NSFW_GIF,Son Dakika Ek Kazanç Videolarımızı İzleyin :\n...,Try My New Game..! :)\nhttps://play.google.com...,**HD** Stream[English Tiwtch](http://www.zifoo...,/r/krat0m <-subscribe for free kilo of strong ...
3,2032,If he'd cheat on you he'll cheat on her. She's...,No spoilers: Do not reveal important details t...,relationships,And now Jessica is gone. They really need to ...,"Probably a meme at this point, particularly th...","hahah. Please, Barry changed his name to hide ...",I found the full video.\nThere were a few peop...
4,2033,I think I remember reading somewhere that tech...,No legal advice: Do not offer or request legal...,AskReddit,"I mean, even if they didn't have anything sign...","Fire her. You can't be sued for it, only the c...",Personally. I'd be wrong to tell you suicide i...,"I call it salary, perhaps a better term is sch..."


In [14]:
print("\nTraining Set")
print(f"Total samples: {len(train_df)}")
print(f"Class distribution:\n{train_df[label_col].value_counts()}")
print(f"Class ratio: {train_df[label_col].value_counts(normalize=True)}")
print(f"\nUnique rules in train: {train_df[rule_col].nunique()}")
print(f"Rules: {train_df[rule_col].unique()}")
print(f"\nUnique subreddits in train: {train_df['subreddit'].nunique()}")

print("\nTest Set")
print(f"Total samples: {len(test_df)}")
print(f"\nUnique rules in test: {test_df[rule_col].nunique()}")
print(f"Rules: {test_df[rule_col].unique()}")
print(f"\nUnique subreddits in test: {test_df['subreddit'].nunique()}")

print("\nAnalysis")
train_rules = set(train_df[rule_col].unique())
test_rules = set(test_df[rule_col].unique())
print(f"Rules in test but NOT in train: {test_rules - train_rules}")
print(f"Number of test samples with unseen rules: {test_df[~test_df[rule_col].isin(train_rules)].shape[0]}")
print(f"Percentage: {100 * test_df[~test_df[rule_col].isin(train_rules)].shape[0] / len(test_df):.2f}%")

train_subreddits = set(train_df['subreddit'].unique())
test_subreddits = set(test_df['subreddit'].unique())


Training Set
Total samples: 2029
Class distribution:
rule_violation
1    1031
0     998
Name: count, dtype: int64
Class ratio: rule_violation
1    0.508132
0    0.491868
Name: proportion, dtype: float64

Unique rules in train: 2
Rules: ['No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.'
 'No legal advice: Do not offer or request legal advice.']

Unique subreddits in train: 100

Test Set
Total samples: 54059

Unique rules in test: 6
Rules: ['No promotion of illegal activity: Do not encourage or promote illegal activities, such as drug-related activity, violence, exploitation, theft, or other criminal behavior.'
 'No medical advice: Do not offer or request specific medical advice, diagnoses, or treatment recommendations.'
 'No Advertising: Spam, referral links, unsolicited advertising, and promotional content are not allowed.'
 "No spoilers: Do not reveal important details that would limit people's ability to enjoy a show or movie."

During our data analysis we noticed a few pain points in solving this problem. First, we have a small training dataset of 2,029 samples, and we are evaluating this on a test set of 54,059 samples. Secondly, we have an additional 4 rules in our test dataset that are unseen in our training dataset, these unseen rules make up 66.76% of our test samples

# Training Helper Functions

In [15]:
if torch.cuda.is_available():
    device = "cuda"
elif platform.system() == "Darwin" and torch.backends.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print(f"Using device: {device}")

Using device: cuda


In [16]:
model = SentenceTransformer("./e5-large-v2-triplet", device="cuda")

def embed_batch(texts):
    return model.encode(
        texts,
        normalize_embeddings=True,
        show_progress_bar=True
    )

In [17]:
def load_or_create_embeddings(df, prefix, cache_dir="embeddings_cache_triplet"):
    os.makedirs(cache_dir, exist_ok=True)
    embeddings = {}

    to_embed = {
        'body': body_col,
        'pos1': 'positive_example_1',
        'pos2': 'positive_example_2',
        'neg1': 'negative_example_1',
        'neg2': 'negative_example_2'
    }

    for key, col in to_embed.items():
        cache_file = f"{cache_dir}/{prefix}_{key}_emb.pkl"

        if os.path.exists(cache_file):
            print(f"Loading cached {prefix} {key} embeddings...")
            embeddings[f"{key}_emb"] = pickle.load(open(cache_file, 'rb'))
        else:
            print(f"Embedding {prefix} {col}...")
            embeddings[f"{key}_emb"] = embed_batch(df[col].tolist())
            pickle.dump(embeddings[f"{key}_emb"], open(cache_file, 'wb'))

    return embeddings



In [18]:

def extract_text_features(text):
    num_chars = len(text)
    num_words = len(text.split())

    has_url = 1 if re.search(r'http[s]?://|www\.', text) else 0

    if num_chars > 0:
        caps_ratio = sum(1 for c in text if c.isupper()) / num_chars
    else:
        caps_ratio = 0

    return np.array([num_chars, num_words, has_url, caps_ratio])

In [19]:
def combine_features(body, rule, pos1, pos2, neg1, neg2, text, subreddit_encoded):
    sim_pos1 = (body * pos1).sum()
    sim_pos2 = (body * pos2).sum()
    sim_neg1 = (body * neg1).sum()
    sim_neg2 = (body * neg2).sum()
    sim_rule = (body * rule).sum()

    pos_avg = (sim_pos1 + sim_pos2) / 2
    neg_avg = (sim_neg1 + sim_neg2) / 2
    pos_neg_ratio = pos_avg / (neg_avg + 1e-6)
    pos_neg_diff = pos_avg - neg_avg

    text_feats = extract_text_features(text)

    return np.concatenate([
        [sim_pos1, sim_pos2, sim_neg1, sim_neg2, sim_rule],  # 5 features
        [pos_neg_ratio, pos_neg_diff],  # 2 features
        text_feats,  # 4 features (num_chars, num_words, has_url, caps_ratio)
        [subreddit_encoded]  # 1 feature
    ])

In [25]:
print("Embedding unique rules...")
all_unique_rules = pd.concat([train_df[rule_col], test_df[rule_col]]).unique()
unique_rule_emb = embed_batch(all_unique_rules.tolist())
rule_to_emb = dict(zip(all_unique_rules, unique_rule_emb))
rule_emb = np.array([rule_to_emb[rule] for rule in train_df[rule_col]])

print("\nEmbedding train set")
train_emb = load_or_create_embeddings(train_df, prefix='train')
body_emb = train_emb['body_emb']
pos1_emb = train_emb['pos1_emb']
pos2_emb = train_emb['pos2_emb']
neg1_emb = train_emb['neg1_emb']
neg2_emb = train_emb['neg2_emb']
rule_emb_test = np.array([rule_to_emb[rule] for rule in test_df[rule_col]])

print("\nEmbedding test set")
test_emb = load_or_create_embeddings(test_df, prefix='test')
body_emb_test = test_emb['body_emb']
pos1_emb_test = test_emb['pos1_emb']
pos2_emb_test = test_emb['pos2_emb']
neg1_emb_test = test_emb['neg1_emb']
neg2_emb_test = test_emb['neg2_emb']

Embedding unique rules...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]


Embedding train set
Loading cached train body embeddings...
Loading cached train pos1 embeddings...
Loading cached train pos2 embeddings...
Loading cached train neg1 embeddings...
Loading cached train neg2 embeddings...

Embedding test set
Loading cached test body embeddings...
Loading cached test pos1 embeddings...
Loading cached test pos2 embeddings...
Loading cached test neg1 embeddings...
Loading cached test neg2 embeddings...


In [28]:
subreddit_encoder = LabelEncoder()
subreddit_encoder.fit(pd.concat([train_df['subreddit'], test_df['subreddit']]))

LabelEncoder()

In [30]:
train_subreddit_encoded = subreddit_encoder.transform(train_df['subreddit'])
y = train_df[label_col].values

X = np.array([
    combine_features(
        body_emb[i], rule_emb[i], 
        pos1_emb[i], pos2_emb[i], 
        neg1_emb[i], neg2_emb[i],
        train_df[body_col].iloc[i],
        train_subreddit_encoded[i]
    ) 
    for i in range(len(train_df))
])


print(f"Feature shape: {X.shape}")
X.shape, y.shape

Feature shape: (2029, 12)


((2029, 12), (2029,))

In [32]:
test_subreddit_encoded = subreddit_encoder.transform(test_df['subreddit'])

X_test = np.array([
    combine_features(
        body_emb_test[i], rule_emb_test[i],
        pos1_emb_test[i], pos2_emb_test[i],
        neg1_emb_test[i], neg2_emb_test[i],
        test_df[body_col].iloc[i],
        test_subreddit_encoded[i]
    )
    for i in range(len(test_df))
])

print(f"Test features shape: {X_test.shape}")

Test features shape: (54059, 12)


# Main Training Loop

In [36]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, n_jobs=-1, class_weight="balanced"),
    'Random Forest': RandomForestClassifier(n_estimators=200, max_depth=10, n_jobs=-1, class_weight="balanced", random_state=42),
    'Gradient Boosting': GradientBoostingClassifier(n_estimators=200, max_depth=5, learning_rate=0.1, random_state=42),
    'XGBoost': XGBClassifier(n_estimators=200, max_depth=4, learning_rate=0.05, subsample=0.8, colsample_bytree=0.8, reg_alpha=1.0, reg_lambda=1.0, random_state=42, n_jobs=-1, eval_metric='logloss'),
    'SVM (RBF)': SVC(kernel='rbf', probability=True, class_weight='balanced', random_state=42)
}

results = {}

for name, model in models.items():
    print(f"Training {name}...")

    model.fit(X, y)
    
    test_preds = model.predict(X_test)
    test_probs = model.predict_proba(X_test)[:,1]
    
    test_results = test_df[["row_id"]].copy()
    test_results["prediction"] = test_probs
    test_results = test_results.merge(solution_df[["row_id", "rule_violation"]], on="row_id", how="left")
    y_test = test_results["rule_violation"].values
    
    print(f"Metrics on test dataset")
    print(classification_report(y_test, test_preds))
    test_auc = roc_auc_score(y_test, test_probs)
    test_acc = (test_preds == y_test).mean()
    print(f"AUC: {test_auc:.4f}")
    print(f"Accuracy: {test_acc:.4f}")

    
    results[name] = {
        'test_auc': test_auc,
        'test_acc': test_acc,
        'model': model,
        'test_probs': test_probs
    }
    print("\n")

print("Summary")
for name, res in results.items():
    print(f"{name:20s} | Test AUC: {res['test_auc']:.4f} | Test Acc: {res['test_acc']:.4f}")

# best_model_name = max(results.items(), key=lambda x: x[1]['test_auc'])[0]
# best_probs = results[best_model_name]['test_probs']
# test[["row_id"]].assign(prediction=best_probs).to_csv("submission.csv", index=False)
# print(f"\nSaved submission.csv using {best_model_name}")

Training Logistic Regression...
Metrics on test dataset
              precision    recall  f1-score   support

           0       0.73      0.75      0.74     28789
           1       0.71      0.68      0.70     25270

    accuracy                           0.72     54059
   macro avg       0.72      0.72      0.72     54059
weighted avg       0.72      0.72      0.72     54059

AUC: 0.7708
Accuracy: 0.7202


Training Random Forest...
Metrics on test dataset
              precision    recall  f1-score   support

           0       0.75      0.37      0.49     28789
           1       0.54      0.86      0.67     25270

    accuracy                           0.60     54059
   macro avg       0.65      0.61      0.58     54059
weighted avg       0.66      0.60      0.58     54059

AUC: 0.7573
Accuracy: 0.5988


Training Gradient Boosting...
Metrics on test dataset
              precision    recall  f1-score   support

           0       0.74      0.32      0.45     28789
           1   